In [13]:
pip install pandas

2228.41s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Using cached pandas-2.3.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (91 kB)
  Using cached numpy-2.3.5-cp312-cp312-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
Using cached pandas-2.3.3-cp312-cp312-macosx_11_0_arm64.whl (10.7 MB)
Using cached numpy-2.3.5-cp312-cp312-macosx_14_0_arm64.whl (5.1 MB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pandas]2m2/3 [pandas]
Note: you may need to restart the kernel to use updated packages.


In [15]:
import requests
from bs4 import BeautifulSoup
import time
import sqlite3
import random

# データベースの設定
DB_FILE = 'github_repos.db'

def create_database():
    """データベースとテーブルを作成する"""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    
    # リポジトリ情報を保存するテーブルを作成
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS repositories (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        repo_name TEXT,
        stars INTEGER,
        language TEXT,
        scrape_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    ''')
    
    conn.commit()
    conn.close()
    print("データベースとテーブルを作成しました")

def scrape_github_page(page_num):
    """指定されたページ番号のGitHubリポジトリ検索結果をスクレイピングする"""
    # 各リクエスト前に1秒待機（指定通り）
    time.sleep(1)
    
    url = f"https://github.com/search?q=google&type=repositories&p={page_num}"
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers)
        
        if response.status_code != 200:
            print(f"エラー: ページ {page_num} - ステータスコード {response.status_code}")
            return []
        
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # リポジトリコンテナを検出（新旧デザイン対応）
        repo_items = []
        
        # 新デザイン用
        new_design = soup.select('div[data-testid="results-list"] > div')
        if new_design:
            repo_items = new_design
            print(f"ページ {page_num} で {len(repo_items)} 件のリポジトリを検出")
        else:
            # 従来のデザイン用
            old_design = soup.select('ul.repo-list > li')
            if old_design:
                repo_items = old_design
                print(f"ページ {page_num} で {len(repo_items)} 件のリポジトリを検出")
        
        if not repo_items:
            print(f"ページ {page_num} にリポジトリが見つかりませんでした。")
            return []
        
        repos = []
        for repo in repo_items:
            # リポジトリ名を取得（複数のセレクタを試す）
            repo_name = None
            repo_selectors = [
                'a[data-testid="result-heading-title"]',
                'a.v-align-middle',
                'h3 a',
                'a.text-bold'
            ]
            
            for selector in repo_selectors:
                element = repo.select_one(selector)
                if element:
                    repo_name = element.text.strip()
                    break
            
            # スター数を取得（複数のセレクタを試す）
            stars = None  # デフォルト値を None に設定
            star_selectors = [
                'a[href*="stargazers"]',
                'a.Link--muted [aria-label*="star"]',
                'a.social-count'
            ]
            
            for selector in star_selectors:
                element = repo.select_one(selector)
                if element:
                    star_text = element.text.strip().replace(',', '')
                    try:
                        if 'k' in star_text.lower():
                            stars = int(float(star_text.lower().replace('k', '')) * 1000)
                        else:
                            stars = int(star_text)
                    except ValueError:
                        stars = None  # 変換できない場合は None のまま
                    break
            
            # 言語を取得（複数のセレクタを試す）
            language = None  # デフォルト値を None に設定
            language_selectors = [
                'span[itemprop="programmingLanguage"]',
                'span.color-fg-muted span.d-inline-block',
                'span.repo-language-color + span'
            ]
            
            for selector in language_selectors:
                element = repo.select_one(selector)
                if element:
                    language = element.text.strip()
                    break
            
            # 言語の円形マークの後のテキストを探す
            if language is None:
                lang_circles = repo.select('span.repo-language-color')
                for circle in lang_circles:
                    next_sibling = circle.find_next_sibling()
                    if next_sibling:
                        language = next_sibling.text.strip()
                        break
            
            print(f"リポジトリ: {repo_name}, スター数: {stars}, 言語: {language}")
            
            repos.append({
                'repo_name': repo_name,
                'stars': stars,
                'language': language
            })
        
        return repos
        
    except Exception as e:
        print(f"ページ {page_num} の処理中にエラー: {e}")
        return []

def save_to_database(repos):
    """リポジトリ情報をデータベースに保存する"""
    if not repos:
        return 0
    
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    
    count = 0
    for repo in repos:
        try:
            cursor.execute(
                "INSERT INTO repositories (repo_name, stars, language) VALUES (?, ?, ?)",
                (repo['repo_name'], repo['stars'], repo['language'])
            )
            count += 1
        except sqlite3.Error as e:
            print(f"データベース保存エラー: {e}")
    
    conn.commit()
    conn.close()
    return count

def display_all_from_database():
    """データベースから全てのリポジトリ情報を取得して表示する"""
    conn = sqlite3.connect(DB_FILE)
    cursor = conn.cursor()
    
    print("\n=== データベースに保存された全リポジトリ情報 ===")
    
    # 総レコード数を取得
    cursor.execute("SELECT COUNT(*) FROM repositories")
    total = cursor.fetchone()[0]
    print(f"総レコード数: {total}")
    
    # 全てのレコードを取得
    cursor.execute("SELECT id, repo_name, stars, language, scrape_date FROM repositories")
    
    # 列名を表示
    print("\nID | リポジトリ名 | スター数 | 言語 | 取得日時")
    print("-" * 80)
    
    # 全レコードを表示
    for row in cursor.fetchall():
        id, repo_name, stars, language, scrape_date = row
        
        # NULL値の場合は "NULL" と表示
        if repo_name is None: repo_name = "NULL"
        if stars is None: stars = "NULL"
        if language is None: language = "NULL"
        
        print(f"{id} | {repo_name} | {stars} | {language} | {scrape_date}")
    
    conn.close()

def main():
    """メイン実行関数"""
    # データベースの作成
    create_database()
    
    # 保存されたリポジトリ数をカウント
    total_repos = 0
    
    # 100ページ分のリポジトリ情報を取得
    for page in range(1, 101):
        print(f"\n=== ページ {page}/100 の処理を開始 ===")
        
        # リポジトリ情報のスクレイピング
        repos = scrape_github_page(page)
        
        if not repos:
            print(f"ページ {page} からリポジトリを取得できませんでした。")
            continue
        
        # データベースに保存
        saved = save_to_database(repos)
        total_repos += saved
        print(f"ページ {page} から {saved} 件のリポジトリ情報をデータベースに保存しました")
        
        # ページ間の待機時間を設定（レート制限回避のため）
        wait_time = random.uniform(2, 4)
        print(f"次のページを取得する前に {wait_time:.1f} 秒待機します...")
        time.sleep(wait_time)
    
    # 保存結果の表示
    print(f"\n合計 {total_repos} 件のリポジトリ情報をデータベースに保存しました")
    
    # データベースから全データを表示
    display_all_from_database()

if __name__ == "__main__":
    main()

データベースとテーブルを作成しました

=== ページ 1/100 の処理を開始 ===
ページ 1 で 10 件のリポジトリを検出
リポジトリ: gaowanlu/google, スター数: 341, 言語: None
リポジトリ: google/googletest, スター数: 37500, 言語: None
リポジトリ: google-research/google-research, スター数: 36800, 言語: None
リポジトリ: googleapis/googleapis, スター数: 8300, 言語: None
リポジトリ: NikolaiT/GoogleScraper, スター数: 2800, 言語: None
リポジトリ: PHPGangsta/GoogleAuthenticator, スター数: 2300, 言語: None
リポジトリ: zh-google-styleguide/zh-google-styleguide, スター数: 10900, 言語: None
リポジトリ: opauth/google, スター数: 120, 言語: None
リポジトリ: googlesamples/google-services, スター数: 3200, 言語: None
リポジトリ: Ponderfly/GoogleTranslateIpCheck, スター数: 5000, 言語: None
ページ 1 から 10 件のリポジトリ情報をデータベースに保存しました
次のページを取得する前に 3.0 秒待機します...

=== ページ 2/100 の処理を開始 ===
エラー: ページ 2 - ステータスコード 429
ページ 2 からリポジトリを取得できませんでした。

=== ページ 3/100 の処理を開始 ===
ページ 3 で 10 件のリポジトリを検出
リポジトリ: googleapis/google-cloud-python, スター数: 5200, 言語: None
リポジトリ: google/google-authenticator, スター数: 5300, 言語: None
リポジトリ: priyankavergadia/google-cloud-4-words, スター数: 8200, 言語: None
リポジトリ: T